In [8]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
import tensorflow as tf
import random
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from tensorflow.keras.optimizers import Adam, RMSprop, SGD
from sklearn.preprocessing import StandardScaler
# Load the dataset
df = pd.read_csv("Islamabad_featured_data.csv")
df.head()


,All sky irradiance,Clear sky irradiance,ALLSKY ICI,Temperature,Dew point,Wet bulb,Surface pressure,ALLSKY SFC PAR TOT,ALLSKY SFC UV INDEX,CLRSKY SFC PAR TOT,QV2M
0,3.51,3.46,0.71,14.11,-8.76,2.67,96.39,60.12,0.48,60.32,1.95
1,3.31,3.33,0.66,13.40,-8.78,2.31,96.52,56.69,0.47,57.84,1.95
2,1.16,2.46,0.23,10.69,-3.81,3.44,96.73,21.05,0.19,41.63,3.05
3,3.19,3.17,0.64,10.90,-1.80,4.55,96.64,54.38,0.45,54.98,3.48
4,3.65,3.64,0.73,14.12,-6.95,3.58,96.20,62.85,0.53,63.70,2.32


In [9]:
X = df.drop("All sky irradiance", axis=1)
y = df["All sky irradiance"]

In [10]:
scaler_X = StandardScaler()
X_scaled = scaler_X.fit_transform(df)

scaler_y = StandardScaler()
y_scaled = scaler_y.fit_transform(y.values.reshape(-1, 1)).flatten() 
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y_scaled, test_size = 0.2,shuffle=False)

In [11]:

def build_lstm_model(num_layers, num_units, learning_rate, optimizer, dropout):
    model = Sequential()
    
    # Add LSTM layers
    for _ in range(num_layers):
        model.add(LSTM(units=num_units, input_shape=(X_train.shape[1], 1), return_sequences=True))
        model.add(Dropout(dropout))
    
    # Add the final LSTM layer
    model.add(LSTM(units=num_units))
    model.add(Dropout(dropout))
    
    # Add the output layer
    model.add(Dense(1))
    
    # Set the optimizer based on the provided parameter
    if optimizer == 'adam':
        opt = Adam(learning_rate=learning_rate)
    elif optimizer == 'rmsprop':
        opt = RMSprop(learning_rate=learning_rate)
    elif optimizer == 'sgd':
        opt = SGD(learning_rate=learning_rate)
    else:
        raise ValueError("Invalid optimizer specified.")
    
    # Compile the model
    model.compile(loss='mean_squared_error', optimizer=opt)
    
    return model

# Define the search space for hyperparameters
param_space = {
    'num_layers': [1, 2, 3],
    'num_units': [32, 64, 128, 256],
    'learning_rate': [1e-5, 1e-4, 1e-3, 1e-2],
    'optimizer': ['adam', 'rmsprop', 'sgd'],
    'dropout': [0.0, 0.1, 0.2, 0.3, 0.4, 0.5]
}

# Perform hyperparameter tuning with random search
best_params = None
best_mse = float('inf')

for _ in range(10):  # Number of random trials
    current_params = {
        'num_layers': random.choice(param_space['num_layers']),
        'num_units': random.choice(param_space['num_units']),
        'learning_rate': random.choice(param_space['learning_rate']),
        'optimizer': random.choice(param_space['optimizer']),
        'dropout': random.choice(param_space['dropout'])
    }

    model = build_lstm_model(**current_params)

    # Train the model
    model.fit(X_train, y_train, epochs=10, batch_size=32, verbose=0)

    # Evaluate the model on the validation set
    y_pred = model.predict(X_test)
    mse = mean_squared_error(y_test, y_pred)

    # Update the best hyperparameters if the current configuration is better
    if mse < best_mse:
        best_params = current_params
        best_mse = mse

    print(f"Trial: {_ + 1}, MSE: {mse}, Best MSE: {best_mse}")

# Print the best hyperparameters found during the random search
print("Best Hyperparameters:", best_params)


/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


26/26 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step
Trial: 1, MSE: 0.14359566643103358, Best MSE: 0.14359566643103358


/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


26/26 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step
Trial: 2, MSE: 0.0991614701797639, Best MSE: 0.0991614701797639


/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


...

26/26 ━━━━━━━━━━━━━━━━━━━━ 1s 34ms/step
Trial: 8, MSE: 0.0068249033837835885, Best MSE: 0.0068249033837835885


/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


26/26 ━━━━━━━━━━━━━━━━━━━━ 1s 22ms/step
Trial: 9, MSE: 0.0035131799773845756, Best MSE: 0.0035131799773845756


/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


26/26 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step
Trial: 10, MSE: 0.006799423504121108, Best MSE: 0.0035131799773845756
Best Hyperparameters: {'num_layers': 2, 'num_units': 256, 'learning_rate': 0.01, 'optimizer': 'rmsprop', 'dropout': 0.1}
